# 📊 02_content_analysis.ipynb

Phân tích nội dung các bài đăng của Coke, Pepsi và Fanta từ tháng 11/2024 đến tháng 3/2025.

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
import string
nltk.download('stopwords')

In [9]:
# Load dữ liệu đã crawl từ bước 1
coke = pd.read_csv("../data/crawl_coke_data.csv")
pepsi = pd.read_csv("../data/crawl_pepsi_data.csv")
fanta = pd.read_csv("../data/crawl_fanta_data.csv")

coke['brand'] = 'Coke'
pepsi['brand'] = 'Pepsi'
fanta['brand'] = 'Fanta'

df = pd.concat([coke, pepsi, fanta])
df['post_date'] = pd.to_datetime(df['post_date'])

## 1. Tổng số bài đăng mỗi tháng

In [10]:
df['month'] = df['post_date'].dt.to_period('M')
monthly_posts = df.groupby(['brand', 'month']).size().unstack(fill_value=0).T

monthly_posts.plot(kind='bar', figsize=(12,6))
plt.title("Số bài đăng mỗi tháng theo thương hiệu")
plt.ylabel("Số bài")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 2. Phân tích độ dài bài viết

In [11]:
df['post_length'] = df['content'].apply(lambda x: len(str(x).split()))

plt.figure(figsize=(10,5))
sns.boxplot(data=df, x='brand', y='post_length')
plt.title("Độ dài bài đăng theo thương hiệu")
plt.ylabel("Số từ")
plt.show()

## 3. Word Cloud theo thương hiệu

In [12]:
stop_words = set(stopwords.words('english') + list(string.punctuation))

def clean_text(text):
    tokens = str(text).lower().split()
    return ' '.join([word for word in tokens if word not in stop_words])

In [13]:
for brand in df['brand'].unique():
    brand_text = df[df['brand'] == brand]['content'].apply(clean_text).str.cat(sep=' ')
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(brand_text)
    
    plt.figure(figsize=(10,5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f"Word Cloud - {brand}")
    plt.show()

## 4. Lưu dữ liệu phân tích
- Dữ liệu này sẽ dùng cho bước 03: brand_comparison.ipynb

In [14]:
df.to_csv("../data/content_analysis.csv", index=False)
print("✅ Content analysis complete and saved!")